# Python私房手册-pydantic

- [pydantic1.71中文文档](https://blog.csdn.net/swinfans/article/details/89629641)

## 模型

### ORM模式

pydantic可以直接解析类似sqlalchemy的模型：

In [53]:
from typing import List
from sqlalchemy import Column, Integer, String
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext.declarative import declarative_base
from pydantic import BaseModel, constr

Base = declarative_base()


class CompanyOrm(Base):
    __tablename__ = 'companies'
    id = Column(Integer, primary_key=True, nullable=False)
    public_key = Column(String(20), index=True, nullable=False, unique=True)
    name = Column(String(63), unique=True)
    domains = Column(ARRAY(String(255)))


class CompanyModel(BaseModel):
    id: int
    public_key: constr(max_length=20)
    name: constr(max_length=63)
    domains: List[constr(max_length=255)]

    class Config:
        orm_mode = True


co_orm = CompanyOrm(
    id=123,
    public_key='foobar',
    name='Testing',
    domains=['example.com', 'foobar.com'],
)

co_model = CompanyModel.from_orm(co_orm)
co_model

CompanyModel(id=123, public_key='foobar', name='Testing', domains=['example.com', 'foobar.com'])

### 错误处理

验证如果没有通过的话，会统一抛出ValidationError错误：

In [1]:
from typing import List
from pydantic import BaseModel, ValidationError, conint


class Location(BaseModel):
    lat = 0.1
    lng = 10.1


class Model(BaseModel):
    is_required: float
    gt_int: conint(gt=42)
    list_of_ints: List[int] = None
    a_float: float = None
    recursive_model: Location = None


data = dict(
    gt_int=21,
    list_of_ints=['1', 2, 'bad'],
    a_float='not a float',
    recursive_model={'lat': 4.2, 'lng': 'New York'},
)

try:
    Model(**data)
except ValidationError as e:
    print(e)

5 validation errors for Model
is_required
  field required (type=value_error.missing)
gt_int
  ensure this value is greater than 42 (type=value_error.number.not_gt; limit_value=42)
list_of_ints -> 2
  value is not a valid integer (type=type_error.integer)
a_float
  value is not a valid float (type=type_error.float)
recursive_model -> lng
  value is not a valid float (type=type_error.float)


除了直接打印错误，还有以下几种方式：
1. e.errors(): 以列表形式返回错误

In [2]:
try:
    Model(**data)
except ValidationError as e:
    print(e.errors())

[{'loc': ('is_required',), 'msg': 'field required', 'type': 'value_error.missing'}, {'loc': ('gt_int',), 'msg': 'ensure this value is greater than 42', 'type': 'value_error.number.not_gt', 'ctx': {'limit_value': 42}}, {'loc': ('list_of_ints', 2), 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}, {'loc': ('a_float',), 'msg': 'value is not a valid float', 'type': 'type_error.float'}, {'loc': ('recursive_model', 'lng'), 'msg': 'value is not a valid float', 'type': 'type_error.float'}]


2. e.json(): 以json形式返回错误

In [3]:
try:
    Model(**data)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "is_required"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "gt_int"
    ],
    "msg": "ensure this value is greater than 42",
    "type": "value_error.number.not_gt",
    "ctx": {
      "limit_value": 42
    }
  },
  {
    "loc": [
      "list_of_ints",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  },
  {
    "loc": [
      "a_float"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  },
  {
    "loc": [
      "recursive_model",
      "lng"
    ],
    "msg": "value is not a valid float",
    "type": "type_error.float"
  }
]


### 定制错误

我们可以自定义一个验证器：

In [4]:
from pydantic import BaseModel, ValidationError, validator


class Model(BaseModel):
    foo: str

    @validator('foo')
    def name_must_contain_space(cls, v):
        if v != 'bar':
            raise ValueError('value must be "bar"')
        return v


try:
    Model(foo='ber')
except ValidationError as e:
    print(e.errors())  # 注意msg和type

[{'loc': ('foo',), 'msg': 'value must be "bar"', 'type': 'value_error'}]


上面我们抛出的是python的内部异常类，我们还可以根据pydantic给的模板类，自己定义一个错误类：

In [16]:
from pydantic import BaseModel, PydanticValueError, PydanticTypeError, ValidationError, validator


class NotABarError(PydanticValueError):
    code = 'not_a_bar'
    msg_template = 'value is not bar, got {wrong_value}'


class Model(BaseModel):
    foo: str

    @validator('foo')
    def name_must_contain_space(cls, v):
        if v != 'bar':
            raise NotABarError(wrong_value=v)
        return v


try:
    Model(foo='ber')
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "foo"
    ],
    "msg": "value is not bar, got ber",
    "type": "value_error.not_a_bar",
    "ctx": {
      "wrong_value": "ber"
    }
  }
]


继承的类主要有PydanticValueError和PydanticTypeError。其中code类属性定义了type，类接受wrong_value的参数，作为ctx的内容。

### 帮助函数

In [17]:
class User(BaseModel):
    name: str
    age: conint(gt=0)
    sex: str
    
    @validator('sex')
    def validate_sex(cls, v):
        if v.lower() not in ['m', 'f', 'male', 'female']:
            raise ValueError(f'{v} is not eligible value')
        return v

#### parse_obj

直接解析字典：

In [26]:
user_d = {
    'name': 'telecomshy',
    'age': 43,
    'sex': 'male'
}

User.parse_obj(user_d)

User(name='telecomshy', age=43, sex='male')

#### parse_raw

In [23]:
import json

user_str = json.dumps(user_d)

解析json字串:

In [24]:
User.parse_raw(user_str)

User(name='telecomshy', age=43, sex='male')

适当设置可以解析pickle数据：

In [28]:
import pickle

user_pickle = pickle.dumps(user_d)
user_pickle

b'\x80\x04\x95.\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x04name\x94\x8c\ntelecomshy\x94\x8c\x03age\x94K+\x8c\x03sex\x94\x8c\x04male\x94u.'

In [29]:
User.parse_raw(user_pickle, content_type='application/pickle', allow_pickle=True)

User(name='telecomshy', age=43, sex='male')

#### parse_file

直接解析文件，实际上就是读入文件然后用`parse_raw`处理：

#### construct

还有一个construct方法，它不会对数据进行验证，优点就是快：

In [37]:
user_d2 = {'name': 'telecomshy', 'age': 43, 'sex': 'ale'}

User.construct(**user_d2)  # sex是错误的，但是没有验证

User(name='telecomshy', age=43, sex='ale')

`construct`有一个`_fields_set`参数，传入一个集合，可以指明哪些字段是需要初始化的，哪些是有默认值的，如果不传入，则模型实例的`__fields_set_`只包含传入的字段。

### 泛型模型

待补充

### 动态模型

有时候，我们需要动态的创建模型，此时可以使用`create_model`方法，以编程方式创建模型：

In [48]:
from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model('DynamicFoobarModel', foo=(str, ...), bar=123)


class StaticFoobarModel(BaseModel):
    foo: str
    bar: int = 123

通过一个元组来定义字段，元组第一个值为类型，第二个值为字段的默认值，如果字段是必填的，使用`...`。

`__base__`参数指定父类：

In [50]:
from pydantic import BaseModel, create_model


class FooModel(BaseModel):
    foo: str
    bar: int = 123


BarModel = create_model(
    'BarModel',
    apple='russet',
    banana='yellow',
    __base__=FooModel,
)

print(BarModel.__fields__.keys())

dict_keys(['foo', 'bar', 'apple', 'banana'])


`__validators__`指定验证器，注意：validator也是动态创建的：

In [51]:
from pydantic import create_model, ValidationError, validator


def username_alphanumeric(cls, v):
    assert v.isalnum(), 'must be alphanumeric'
    return v


validators = {
    'username_validator': validator('username')(username_alphanumeric)
}

UserModel = create_model('UserModel',
                         username=(str, ...),
                         __validators__=validators)

user = UserModel(username='scolvin')

try:
    UserModel(username='scolvi%n')
except ValidationError as e:
    print(e)

1 validation error for UserModel
username
  must be alphanumeric (type=assertion_error)


### 伪不变性

可以通过`allow_mutation = False`将模型配置为不可变的：

In [54]:
from pydantic import BaseModel


class FooBarModel(BaseModel):
    a: str
    b: dict

    class Config:
        allow_mutation = False


foobar = FooBarModel(a='hello', b={'apple': 'pear'})

try:
    foobar.a = 'different'
except TypeError as e:
    print(e)

print(foobar.a)
print(foobar.b)
foobar.b['apple'] = 'grape'  # 不可变不能阻止修改可变对象
print(foobar.b)

"FooBarModel" is immutable and does not support item assignment
hello
{'apple': 'pear'}
{'apple': 'grape'}


#### 字段排序

pydantic字段排序规则为：所有具有注解的字段(无论是只有注解还是具有默认值)都将位于没有注解的字段之前。在各自的组中，字段按照定义时的顺序保存：

In [55]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    a: int
    b = 2
    c: int = 1
    d = 0
    e: float


print(Model.__fields__.keys())

m = Model(e=2, a=1)
print(m.dict())

try:
    Model(a='x', b='x', c='x', d='x', e='x')
except ValidationError as e:
    error_locations = [e['loc'] for e in e.errors()]

print(error_locations)

dict_keys(['a', 'c', 'e', 'b', 'd'])
{'a': 1, 'c': 1, 'e': 2.0, 'b': 2, 'd': 0}
[('a',), ('c',), ('e',), ('b',), ('d',)]


#### 必需字段

要声明一个字段是必需的，可以只添加注释，或者使用`...`：

In [59]:
from pydantic import BaseModel, Field


class Model(BaseModel):
    a: int
    b: int = ...
    c: int = Field(...)

如果要指定一个可以接受None值的必需字段，使用`Optional`。注意，此时a不再是必需的，变成可选的，相当于默认值为None，但是b和c是必需的，即使是None，也要明确的输入：

In [61]:
from typing import Optional
from pydantic import BaseModel, Field, ValidationError


class Model(BaseModel):
    a: Optional[int]
    b: Optional[int] = ...
    c: Optional[int] = Field(...)


print(Model(b=1, c=2))

try:
    Model(a=1, b=2)
except ValidationError as e:
    print(e)

a=None b=1 c=2
1 validation error for Model
c
  field required (type=value_error.missing)


#### 动态默认值

In [1]:
from datetime import datetime
from uuid import UUID, uuid4
from pydantic import BaseModel, Field


class Model(BaseModel):
    uid: UUID = Field(default_factory=uuid4)
    updated: datetime = Field(default_factory=datetime.utcnow)


m1 = Model()
m2 = Model()
print(f'{m1.uid} != {m2.uid}')
print(f'{m1.updated} != {m2.updated}')

841db694-00c1-48d9-a071-049f229a479a != 4d63cfae-d87c-4f90-8b00-6506b4c2e09a
2022-04-13 00:55:02.175128 != 2022-04-13 00:55:02.176129


#### 私有属性

In [5]:
from datetime import datetime
from random import randint

from pydantic import BaseModel, PrivateAttr


class TimeAwareModel(BaseModel):
    _processed_at: datetime = PrivateAttr(default_factory=datetime.now)
    _secret_value: str = PrivateAttr()

    def __init__(self, **data):
        super().__init__(**data)
        self._secret_value = 42


m = TimeAwareModel()
print(m._processed_at)
print(m._secret_value)

2022-04-13 09:02:35.946331
42


可见，传入的参数不影响私有属性的值。在内部，私有属性保存在`__slots__`中：

In [8]:
m = TimeAwareModel(_secret_value=3)
m._secret_value

42

可以设置`Config.underscore_attrs_are_private`为`True`，此时任何非`ClassVar`的下划线属性都将被当做实例的私有属性：

In [9]:
from typing import ClassVar
from pydantic import BaseModel


class Model(BaseModel):
    _class_var: ClassVar[str] = 'class var value'
    _private_attr: str = 'private attr value'

    class Config:
        underscore_attrs_are_private = True


print(Model._class_var)
print(Model._private_attr)
print(Model()._private_attr)

class var value
<member '_private_attr' of 'Model' objects>
private attr value


当使用`ClassVar`声明了为类属性，则不会再初始化为实例属性：

In [13]:
m = Model(_class_var='instance value')
m._class_var

'class var value'

#### parse_obj_as

想象一下我们有这样的数据：

In [14]:
data = [{'id': 1, 'name': 'my item 1'}, {'id': 2, 'name': 'my item 2'}]

如果要解析成模型，可能会想到新建一个列表，然后循环迭代，把data中的每一项转换成模型，添加到新列表中。或者建立一个嵌套的复合模型：

In [22]:
from pydantic import BaseModel, parse_obj_as
from typing import List

class Item(BaseModel):
    id: int
    name: str


class Items(BaseModel):
    items: list[Item]
        

model = Items(items=data)
model

Items(items=[Item(id=1, name='my item 1'), Item(id=2, name='my item 2')])

其实不用这么麻烦，用`parse_obj_as`可以直接解析，其第一个参数为类型注释，第二个为值：

In [23]:
parse_obj_as(List[Item], data)

[Item(id=1, name='my item 1'), Item(id=2, name='my item 2')]

pydantic还包含两个独立函数，`parse_file_as`和`parse_raw_as`，类似`parse_obj_as`，分别解析文件和字符串。

## 字段类型

### 标准库类型

#### Sequence和Iterable

Sequence类型会把传入的对象转换成列表保存，所以如果传入的是生成器，会被消耗掉，如果是无限循环的生成器，则不能设置为Sequence类型。此时可以设置为Iterable类型，Iterable不会转换原始对象，它只会检查对象有没有`__iter__`方法。

这自然而然会产生一个问题，既然Iterable仅仅只检查对象的`__iter__`方法，那它怎么知道这个容器对象包含的值的类型呢？此时需要手动创建一个验证器，来进行验证：

In [8]:
import itertools
from typing import Iterable
from pydantic import BaseModel, validator, ValidationError
from pydantic.fields import ModelField


class Model(BaseModel):
    infinite: Iterable[int]

    @validator('infinite')
    # You don't need to add the "ModelField", but it will help your
    # editor give you completion and catch errors
    # field是值iterable对应的Model中定义的字段对象
    def infinite_first_int(cls, iterable, field: ModelField):
        first_value = next(iterable)
        print(f'field is: {field}')
        if field.sub_fields:
            # The Iterable had a parameter type, in this case it's int
            # We use it to validate the first value
            sub_field = field.sub_fields[0]
            v, error = sub_field.validate(first_value, {}, loc='first_value')
            if error:
                raise ValidationError([error], cls)
        return itertools.chain([first_value], iterable)
    

def infinite_strs():
    while True:
        for letter in 'allthesingleladies':
            yield letter


try:
    Model(infinite=infinite_strs())
except ValidationError as e:
    print(e.errors())

field is: name='infinite' type=Iterable[int] required=True
[{'loc': ('infinite', 'first_value'), 'msg': 'value is not a valid integer', 'type': 'type_error.integer'}]


#### Enum和Choice

Enum类型的话，实例化的时候传入的要是枚举的值，不能传入键：

In [30]:
from enum import Enum, IntEnum

from pydantic import BaseModel, ValidationError


class FruitEnum(str, Enum):
    pear = 'pear'
    banana = 'banana'


class ToolEnum(IntEnum):
    spanner = 1
    wrench = 2


class CookingModel(BaseModel):
    fruit: FruitEnum = FruitEnum.pear
    tool: ToolEnum = ToolEnum.spanner


print(CookingModel())
print(CookingModel(tool=2, fruit='banana'))

try:
    CookingModel(fruit='other')
except ValidationError as e:
    print(e)

fruit=<FruitEnum.pear: 'pear'> tool=<ToolEnum.spanner: 1>
fruit=<FruitEnum.banana: 'banana'> tool=<ToolEnum.wrench: 2>
1 validation error for CookingModel
fruit
  value is not a valid enumeration member; permitted: 'pear', 'banana' (type=type_error.enum; enum_values=[<FruitEnum.pear: 'pear'>, <FruitEnum.banana: 'banana'>])


#### Type

Type表示只能是某个类的子类，比如下面`Type[Foo]`，表示`just_subclasses`字段只能是Foo类及其子类。如果只是`Type`，表示可以是任意类，注意不能是实例：

In [24]:
from typing import Type

from pydantic import BaseModel
from pydantic import ValidationError


class Foo:
    pass


class Bar(Foo):
    pass


class Other:
    pass


class SimpleModel(BaseModel):
    just_subclasses: Type[Foo]


SimpleModel(just_subclasses=Foo)
SimpleModel(just_subclasses=Bar)
try:
    SimpleModel(just_subclasses=Other)
except ValidationError as e:
    print(e)

1 validation error for SimpleModel
just_subclasses
  subclass of Foo expected (type=type_error.subclass; expected_class=Foo)


#### Literal类型

Literal是3.8引入的typing新类型，有点类似Enum，不过是个轻量级的多选一，比如下面的模型，表示`flavor`只能是`apple`或者`pumpkun`：

In [33]:
from typing import Literal

from pydantic import BaseModel, ValidationError


class Pie(BaseModel):
    flavor: Literal['apple', 'pumpkin']


Pie(flavor='apple')
Pie(flavor='pumpkin')
try:
    Pie(flavor='cherry')
except ValidationError as e:
    print(str(e))

1 validation error for Pie
flavor
  unexpected value; permitted: 'apple', 'pumpkin' (type=value_error.const; given=cherry; permitted=('apple', 'pumpkin'))


#### 约束类型

pydantic提供`con*`类型函数来约束许多常见类型的值，相当于是为常见类型提供了验证，注意`Field`类型，相当于一个通用的验证器：

In [31]:
from decimal import Decimal

from pydantic import (
    BaseModel,
    NegativeFloat,
    NegativeInt,
    PositiveFloat,
    PositiveInt,
    conbytes,
    condecimal,
    confloat,
    conint,
    conlist,
    conset,
    constr,
    Field,
)


class Model(BaseModel):
    short_bytes: conbytes(min_length=2, max_length=10)
    strip_bytes: conbytes(strip_whitespace=True)

    short_str: constr(min_length=2, max_length=10)
    regex_str: constr(regex=r'^apple (pie|tart|sandwich)$')
    strip_str: constr(strip_whitespace=True)

    big_int: conint(gt=1000, lt=1024)
    mod_int: conint(multiple_of=5)  # 5的倍数
    pos_int: PositiveInt
    neg_int: NegativeInt

    big_float: confloat(gt=1000, lt=1024)
    unit_interval: confloat(ge=0, le=1)
    mod_float: confloat(multiple_of=0.5)
    pos_float: PositiveFloat
    neg_float: NegativeFloat

    short_list: conlist(int, min_items=1, max_items=4)
    short_set: conset(int, min_items=1, max_items=4)

    decimal_positive: condecimal(gt=0)
    decimal_negative: condecimal(lt=0)
    decimal_max_digits_and_places: condecimal(max_digits=2, decimal_places=2)
    mod_decimal: condecimal(multiple_of=Decimal('0.25'))

    bigger_int: int = Field(..., gt=10000)

`conint`类型可以传入一个`strict=True`的参数，表示不进行类型的转换，相当于直接使用`StrcitInt`类型，默认情况下，字段类型为float，传入int值会转换成float，如果设置了`strict=True`，则会报错。

#### ByteSize

ByteSize挺有意思，专门用来表示磁盘容量大小，并且有个`human_readable`方法，可以在不同的单位之间进行转换：

In [38]:
from pydantic import BaseModel, ByteSize


class DiskSize(BaseModel):
    size: ByteSize


disk_a_size = DiskSize(size='50GB')
disk_a_size.size  # 单位是字节

50000000000

默认是GiB，和GB的差异可以参考这篇文章：
- [GB 和 GiB 的区别](https://www.cnblogs.com/miracle-luna/p/11275414.html)

简单来说，以前我们说的1024，实际上是iB的进制，1000是B的进制。

In [42]:
disk_a_size.size.human_readable()

'46.6GiB'

In [44]:
disk_a_size.size.to('TB')

0.05

#### 自定义类型

##### 带有 `__get_validators__ `的类

我们甚至还可以自定义数据类型，首先定义一个具有`__get_validators__`类方法的类，通过它调用相应的验证器：

In [19]:
import re
from pydantic import BaseModel
from pprint import pprint

# https://en.wikipedia.org/wiki/Postcodes_in_the_United_Kingdom#Validation
post_code_regex = re.compile(
    r'(?:'
    r'([A-Z]{1,2}[0-9][A-Z0-9]?|ASCN|STHL|TDCU|BBND|[BFS]IQQ|PCRN|TKCA) ?'
    r'([0-9][A-Z]{2})|'
    r'(BFPO) ?([0-9]{1,4})|'
    r'(KY[0-9]|MSR|VG|AI)[ -]?[0-9]{4}|'
    r'([A-Z]{2}) ?([0-9]{2})|'
    r'(GE) ?(CX)|'
    r'(GIR) ?(0A{2})|'
    r'(SAN) ?(TA1)'
    r')'
)


class PostCode(str):
    """
    Partial UK postcode validation. Note: this is just an example, and is not
    intended for use in production; in particular this does NOT guarantee
    a postcode exists, just that it has a valid format.
    """

    @classmethod
    def __get_validators__(cls):
        # one or more validators may be yielded which will be called in the
        # order to validate the input, each validator will receive as an input
        # the value returned from the previous validator
        yield cls.validate

    @classmethod
    def __modify_schema__(cls, field_schema):
        # __modify_schema__ should mutate the dict it receives in place,
        # the returned value will be ignored
        field_schema.update(
            # simplified regex here for brevity, see the wikipedia link above
            pattern='^[A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2}$',
            # some example postcodes
            examples=['SP11 9DG', 'w1j7bu'],
        )

    @classmethod
    def validate(cls, v):
        if not isinstance(v, str):
            raise TypeError('string required')
        m = post_code_regex.fullmatch(v.upper())
        if not m:
            raise ValueError('invalid postcode format')
        # you could also return a string here which would mean model.post_code
        # would be a string, pydantic won't care but you could end up with some
        # confusion since the value's type won't match the type annotation
        # exactly
        return cls(f'{m.group(1)} {m.group(2)}')  # 这里实际上调用的是str的构造函数

    def __repr__(self):
        return f'PostCode({super().__repr__()})'


class Model(BaseModel):
    post_code: PostCode


model = Model(post_code='sw8 5el')
print(model)
print(model.post_code)
print(Model.schema_json(indent=4))

post_code=PostCode('SW8 5EL')
SW8 5EL
{
    "title": "Model",
    "type": "object",
    "properties": {
        "post_code": {
            "title": "Post Code",
            "pattern": "^[A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2}$",
            "examples": [
                "SP11 9DG",
                "w1j7bu"
            ],
            "type": "string"
        }
    },
    "required": [
        "post_code"
    ]
}


##### 允许任意类型

甚至普通的类也可以，由于此时没有定义验证器，因此只会判断是不是该类的实例。普通类的话，一定要在Config中设置`arbitrary_types_allowed = True`：

In [22]:
from pydantic import BaseModel, ValidationError


# This is not a pydantic model, it's an arbitrary class
class Pet:
    def __init__(self, name: str):
        self.name = name


class Model(BaseModel):
    pet: Pet
    owner: str

    class Config:
        arbitrary_types_allowed = True

In [25]:
pet = Pet(name='Hedwig')
model = Model(owner='Harry', pet=pet)
print(model)
print(model.pet)
print(model.pet.name)
print(type(model.pet))

pet=<__main__.Pet object at 0x00000200228034F0> owner='Harry'
Hedwig
<class '__main__.Pet'>


In [26]:
try:
    # If the value is not an instance of the type, it's invalid
    Model(owner='Harry', pet='Hedwig')
except ValidationError as e:
    print(e)

1 validation error for Model
pet
  instance of Pet expected (type=type_error.arbitrary_type; expected_arbitrary_type=Pet)


##### 泛型类作为类型

待补充

### Field字段

前面已经见识过Field字段，这里详细加以阐述，Field是一个函数，它可以为字段提供一些额外的信息或者约束，它接受的参数如下：
- default：该参数为位置参数。表示字段的默认值。因为 Field 替换字段的默认值，所以可以使用第一个参数设置默认值。使用省略号表示字段是必需的。★★★★★
- default_factory：一个零参数的可调用对象，当需要该字段的默认值时将调用它。除了其他用途外，它还可用于设置动态默认值。禁止同时设置default和default_factory。★★★★★
- alias：字段的公开名称。★★★★
- title：如果忽略，则使用 filed_name.title()。
- description：如果忽略并且注解是子模型，将会使用子模型的文档字符串。
- const：如果该参数出现，则必须与字段的默认值相同。
- gt：对于数值值 (int，float，Decimal)，将向 JSON 模式添加一个 “大于” 验证和一个 exclusiveMinimum 注解。
- ge：对于数值值 ，将向 JSON 模式添加一个 “大于等于” 验证和一个 minimum 注解。
- lt：对于数值值 ，将向 JSON 模式添加一个"小于" 验证和一个 exclusiveMaximum 注解。
- le：对于数值值 ，将向 JSON 模式添加一个 “小于等于” 验证和一个 maximum 注解。
- multiple_of：对于数值值，将向 JSON 模式添加一个 “倍数” 验证和一个 multipleOf 注解。
- min_items：对于列表值，将向 JSON 模式添加相应的验证和 minItems 注解。
- max_items：对于列表值，将向 JSON 模式添加相应的验证和 maxItems 注解。
- min_length：对于字符串值，将向 JSON 模式添加相应的验证和 minLength 注解。
- max_length：对于字符串值，将向 JSON 模式添加相应的验证和 maxLength 注解。
- regex：对于字符串值，将向 JSON 模式添加一个从传递的字符串生成的正则表达式验证和一个 pattern 注解。

可见，Field也可以对字段进行验证，总结一下，目前我们已经有三种方式可以对字段进行验证：
1. 内置约束类型
2. Field字段
3. 自定义验证器

#### 和约束类型关系

Field字段的约束不能和约束类型一起使用，因为约束类型的优先级高一些，当定义了约束类型，Field字段定义的约束就不会再生效了。此时会抛出相应错误：

In [45]:
try:
    class Model(BaseModel):
        foo: PositiveInt = Field(..., lt=10)
except ValueError as e:
    print(e)

On field "foo" the following field constraints are set but not enforced: lt. 
For more details see https://pydantic-docs.helpmanual.io/usage/schema/#unenforced-field-constraints


但是此时Field字段可以接收类似`exclusiveMaximum`关键字参数，此时会在schema中改约束，但实际上并不生效：

In [46]:
class Model(BaseModel):
    foo: PositiveInt = Field(..., exclusiveMaximum=10)

pprint(Model.schema())

{'properties': {'foo': {'exclusiveMaximum': 10,
                        'exclusiveMinimum': 0,
                        'title': 'Foo',
                        'type': 'integer'}},
 'required': ['foo'],
 'title': 'Model',
 'type': 'object'}


In [48]:
Model(foo=42)  # 小于约束并未生效

Model(foo=42)

## 验证器

用`validator`装饰器可以自定义对值的验证，以下几种情况需要自定义验证器：
1. 复杂的验证逻辑
2. 验证时需要用到其它的字段
3. 动态的默认值

In [45]:
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v


user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)
print(user)

try:
    UserModel(
        name='samuel',
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn2',
    )
except ValidationError as e:
    print(e)

name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'
2 validation errors for UserModel
name
  must contain a space (type=value_error)
password2
  passwords do not match (type=value_error)


被装饰的函数，除了cls和v，还有几个参数：
- values：已验证的其它字段的字段名到字段值的映射。
- config：模型的配置
- field：要验证的字段，注意，是字段不是值。v是值，field是字段对象，可以获取字段的类型注释等属性。

验证器只能抛出`ValueError`、`TypeError`或`AssertionError`三种异常。

### pre和each-item验证器

验证器可以添加很多参数：
- pre: 保证这个验证器在其它验证器之前运行
- each_item: 被装饰的函数里的v是容器字段里面的每一个元素

另外，还有两点要注意：
- `*`表示应用于所有字段
- 验证器可以传递多个字段名称，作用于多个字段

In [7]:
from typing import List
from pydantic import BaseModel, ValidationError, validator


class DemoModel(BaseModel):
    square_numbers: List[int] = []
    cube_numbers: List[int] = []

    @validator('*', pre=True)
    def split_str(cls, v):
        if isinstance(v, str):
            return v.split('|')
        return v

    @validator('cube_numbers', 'square_numbers')
    def check_sum(cls, v):
        if sum(v) > 42:
            raise ValueError('sum of numbers greater than 42')
        return v

    @validator('square_numbers', each_item=True)
    def check_squares(cls, v):
        assert v ** 0.5 % 1 == 0, f'{v} is not a square number'
        return v

    @validator('cube_numbers', each_item=True)
    def check_cubes(cls, v):
        # 64 ** (1 / 3) == 3.9999999999999996 (!)
        # this is not a good way of checking cubes
        assert v ** (1 / 3) % 1 == 0, f'{v} is not a cubed number'
        return v


print(DemoModel(square_numbers=[1, 4, 9]))
print(DemoModel(square_numbers='1|4|16'))
print(DemoModel(square_numbers=[16], cube_numbers=[8, 27]))

try:
    DemoModel(square_numbers=[1, 4, 2])
except ValidationError as e:
    print(e)

try:
    DemoModel(cube_numbers=[27, 27])
except ValidationError as e:
    print(e)

square_numbers=[1, 4, 9] cube_numbers=[]
square_numbers=[1, 4, 16] cube_numbers=[]
square_numbers=[16] cube_numbers=[8, 27]
1 validation error for DemoModel
square_numbers -> 2
  2 is not a square number (type=assertion_error)
1 validation error for DemoModel
cube_numbers
  sum of numbers greater than 42 (type=value_error)


注意，如果包含`each_item`验证器验证的字段不能是父类的字段，此时只能使用普通验证器，`v`是原字段，手动的进行拆分：

In [14]:
from typing import List
from pydantic import BaseModel, ValidationError, validator


class ParentModel(BaseModel):
    names: List[str]


# 这种情况验证器根本不会运行
class ChildModel(ParentModel):
    @validator('names', each_item=True)
    def check_names_not_empty(cls, v):
        assert v != '', 'Empty strings are not allowed.'
        return v

    
try:
    child = ChildModel(names=['Alice', 'Bob', 'Eve', ''])
except ValidationError as e:
    print(e)
else:
    print('No ValidationError caught.')


class ChildModel2(ParentModel):
    @validator('names')
    def check_names_not_empty(cls, v):
        for name in v:
            assert name != '', 'Empty strings are not allowed.'
        return v

print()

try:
    child = ChildModel2(names=['Alice', 'Bob', 'Eve', ''])
except ValidationError as e:
    print(e)

No ValidationError caught.

1 validation error for ChildModel2
names
  Empty strings are not allowed. (type=assertion_error)


### always验证

默认情况下，只有传入了参数才会进行验证，默认值是不验证的，即不会调用验证器：

In [1]:
from datetime import datetime
from pydantic import BaseModel, validator


class DemoModel(BaseModel):
    name: str = 'telecomshy'

    @validator('name')
    def set_ts_now(cls, v):
        print("name's validator is running")
        if v != 'shy':
            raise ValueError('name must be shy')
        return v


print(DemoModel())

name='telecomshy'


如果添加了always，则总是会执行验证器，这可以用在一些需要动态默认值的场合：

In [36]:
from datetime import datetime
from pydantic import BaseModel, validator


class DemoModel(BaseModel):
    ts: datetime = None

    @validator('ts', pre=True, always=True)
    def set_ts_now(cls, v):
        return v or datetime.now()


print(DemoModel())
print(DemoModel(ts='2017-11-08 14:00'))

ts=datetime.datetime(2022, 4, 19, 11, 18, 14, 933904)
ts=datetime.datetime(2017, 11, 8, 14, 0)


### 复用验证器

`allow_reuse`参数可以设定复用验证器：

In [37]:
from pydantic import BaseModel, validator


def normalize(name: str) -> str:
    return ' '.join((word.capitalize()) for word in name.split(' '))


class Producer(BaseModel):
    name: str

    _normalize_name = validator('name', allow_reuse=True)(normalize)


class Consumer(BaseModel):
    name: str

    _normalize_name = validator('name', allow_reuse=True)(normalize)


jane_doe = Producer(name='JaNe DOE')
john_doe = Consumer(name='joHN dOe')
assert jane_doe.name == 'Jane Doe'
assert john_doe.name == 'John Doe'

也可以这样写，更简洁一点：

In [38]:
from pydantic import validator, BaseModel


@validator('name', allow_reuse=True)
def normalize(name: str) -> str:
    return ' '.join((word.capitalize()) for word in name.split(' '))


class Producer(BaseModel):
    name: str

    _normalize_name = normalize


class Consumer(BaseModel):
    name: str

    _normalize_name = normalize


jane_doe = Producer(name='JaNe DOE')
john_doe = Consumer(name='joHN dOe')
assert jane_doe.name == 'Jane Doe'
assert john_doe.name == 'John Doe'

### 根验证器

除了单独的字段，还可以定义`root_validator`根验证器，比如多个字段的之间的关系，比较适合使用根验证器：

In [43]:
from pydantic import BaseModel, ValidationError, root_validator


class UserModel(BaseModel):
    username: str
    password1: str
    password2: str

    @root_validator(pre=True)
    def check_card_number_omitted(cls, values):
        print(values)
        assert 'card_number' not in values, 'card_number should not be included'
        return values

    @root_validator
    def check_passwords_match(cls, values):
        pw1, pw2 = values.get('password1'), values.get('password2')
        if pw1 is not None and pw2 is not None and pw1 != pw2:
            raise ValueError('passwords do not match')
        return values


print(UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn'))

{'username': 'scolvin', 'password1': 'zxcvbn', 'password2': 'zxcvbn'}
username='scolvin' password1='zxcvbn' password2='zxcvbn'


In [40]:
try:
    UserModel(username='scolvin', password1='zxcvbn', password2='zxcvbn2')
except ValidationError as e:
    print(e)

1 validation error for UserModel
__root__
  passwords do not match (type=value_error)


In [44]:
try:
    UserModel(
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn',
        card_number='1234',
    )
except ValidationError as e:
    print(e)

{'username': 'scolvin', 'password1': 'zxcvbn', 'password2': 'zxcvbn', 'card_number': '1234'}
1 validation error for UserModel
__root__
  card_number should not be included (type=assertion_error)


其中values是一个包含字段名和字段值的字典。根验证器的`pre=True`表示在所有字段验证之前运行，默认是`pre=False`。

如果`pre=True`时根验证器引发错误，则不会进行字段验证。否则即使之前的验证器失败，默认情况下也会调用根验证器，可以通过为验证器设置 `skip_on_failure=True`关键字参数来改变这种行为。

### 字段检查

默认情况下，当定义了一个验证器，创建类的时候，会检查验证器相关的字段是否存在。但是有时候，我们继承了一个模型，要验证继承模型的字段，则可以设置`check_fields=False`，不抛出错误。

In [48]:
from datetime import datetime
from pydantic import BaseModel, validator


try:
    class DemoModel(BaseModel):

        @validator('ts', pre=True, always=True)
        def set_ts_now(cls, v):
            return v or datetime.now()
except Exception as e:
    print(e)

Validators defined with incorrect fields: set_ts_now (use check_fields=False if you're inheriting from the model and intended this)


测试发现，继承模型的话，不设置`check_feilds`貌似也不会抛出错误，不知道这个功能可以应用在哪里：

In [63]:
class DemoModel(BaseModel):
    ts: datetime = None
    

class SubDemo(DemoModel):
    
    @validator('ts', pre=True, always=True)
    def set_ts_now(cls, v):
        return v or datetime.now()

## 模型配置

可以在模型内定义Config类对模型进行配置，可配置项如下：
- title：声成的 JSON 模式的标题。
- anystr_strip_whitespace：是否移除 str 和 bytes 类型中前导和尾随的空白字符(默认值为 False)。★★★★★
- min_anystr_length：str 和 bytes 类型的最小长度 (默认为 0)。★★★★★
- max_anystr_length：str 和 bytes 类型的最小长度 (默认为 2 ** 16)。★★★★★
- validate_all：是否验证字段的默认值 (默认为 False)。★★★
- extra：在模型初始化时是否忽略、允许或禁止额外的属性。可接受字符串值 ignore、allow 或 forbid 以及 Extra 枚举 (例如， Extra.ignore)。如果模型包含了额外的属性，则 forbid 将会导致验证失败。ignore 将静默忽略任何额外属性。allow 将属性分配给模型。★★★
- allow_mutation：模型是否是伪不可变的。例如，是否允许`__setattr__`(默认为 True)。
- use_enum_values：是否使用枚举的 value 属性而不是原始枚举填充模型。如果之后想要序列化 model.dict() ，这可能会很有用 (默认值为 False)。★★★
- fields：包含每个字段的模式信息的字典； 这等效于使用Field类(默认为 None)。
- validate_assignment：是否对属性的赋值执行验证 (默认为 False)。
- allow_population_by_field_name：是否可以用模型属性给出的名称填充别名字段，以及别名 (默认为 False )。
- error_msg_templates：用于覆盖默认错误消息模板的字典。传入一个字典，其中的键与您想要覆盖的错误消息相匹配 (默认为 {})。
- arbitrary_types_allowed：是否允许字段使用任意用户类型(只需检查值是否为该类型的实例即可对它们进行验证)。如果为 False，则会在模型声明时引发RuntimeError(默认为 False)。
- orm_mode：是否允许使用ORM模式。★★★★★
- getter_dict：一个自定义类 (应该继承自 GetterDict)，用于分解ORM类进行验证，并与orm_mode一起使用。
- alias_generator：接受字段名并返回其别名的可调用对象。
- keep_untouched：模型的默认值的类型元组 (例如，描述符)，该默认值在模型创建期间不应更改，也不会包含在模型模式中。注意：这意味着模型上具有此类型的默认值的属性 (而不是此类型的注释) 将被保留。
- schema_extra：用于拓展/更新生成的JSON 模式的字典，或用于对其进行后处理(post-process)的可调用对象。参见模式定制。
- json_loads：用于解码 JSON 的自定义函数。参见自定义JSON反序列化。
- json_dumps：用于编码 JSON的自定义函数。参见自定义JSON反序列化。
- json_encoders：用于自定义类型被编码成JSON的方式的字典。参见JSON序列化。
- underscore_attrs_are_private：是否将任何下划线非类 (non-class) 变量属性当做私有属性，或让它们保持原样。参见私有模型属性。

In [11]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    v: str

    class Config:
        max_anystr_length = 10
        error_msg_templates = {
            'value_error.any_str.max_length': 'max_length:{limit_value}',
        }


try:
    Model(v='x' * 20)
except ValidationError as e:
    print(e)

1 validation error for Model
v
  max_length:10 (type=value_error.any_str.max_length; limit_value=10)


### 别名生成器

可以通过`alias_generator`指定一个函数以对名称进行处理，注意和别名的区别，我们可以通过`Field`字段或者在`Config`中通过`fields`设置别名，而别名生成器对所有名称进行处理，这个函数也可以放在`Config`类内部，此时是一个类方法：

In [15]:
from pydantic import BaseModel


def to_camel(string: str) -> str:
    return ''.join(word.capitalize() for word in string.split('_'))


class Voice(BaseModel):
    name: str
    language_code: str

    class Config:
        alias_generator = to_camel


voice = Voice(Name='Filiz', LanguageCode='tr-TR')
print(voice.language_code)
print(voice.dict(by_alias=True))

tr-TR
{'Name': 'Filiz', 'LanguageCode': 'tr-TR'}


### 别名优先级

生成别名有很多种方式，其优先级为：
- 在模型上直接通过`Field(..., alias=<alias>)`设置。
- 在模型上的`Config.fields`中定义。
- 在父模型上通过`Field(..., alias=<alias>)`。
- 在父模型上的`Config.fields`中定义。
- 由`alias_generator`生成，无论它是在模型上还是在父模型上。

In [21]:
from pydantic import BaseModel, Field
from pprint import pprint


class Voice(BaseModel):
    name: str = Field(None, alias='ActorName')  # 可以通过
    language_code: str = None
    mood: str = None


class Character(Voice):
    act: int = 1

    class Config:
        # 通过fields设置别名
        fields = {'language_code': 'lang'}
        
        @classmethod
        def alias_generator(cls, string: str) -> str:
            return ''.join(word.capitalize() for word in string.split('_'))


# schema可以打印模型的信息
pprint(Character.schema(by_alias=True))

{'properties': {'Act': {'default': 1, 'title': 'Act', 'type': 'integer'},
                'ActorName': {'title': 'Actorname', 'type': 'string'},
                'Mood': {'title': 'Mood', 'type': 'string'},
                'lang': {'title': 'Lang', 'type': 'string'}},
 'title': 'Character',
 'type': 'object'}


## schema

有些中文教程把schema翻译成模式，但是我觉得难以理解。schema其实是用来描述模型结构的字典，或者json...pydantic可以方便的生成不同格式的schema：

In [25]:
from enum import Enum
from pydantic import BaseModel, Field


class FooBar(BaseModel):
    count: int
    size: float = None


class Gender(str, Enum):
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'


class MainModel(BaseModel):
    foo_bar: FooBar = Field(...)
    gender: Gender = Field(None, alias='Gender')
    snap: int = Field(
        42,
        title='The Snap',
        description='this is the value of snap',
        gt=30,
        lt=50,
    )

    class Config:
        title = 'Main'


pprint(MainModel.schema())

{'definitions': {'FooBar': {'properties': {'count': {'title': 'Count',
                                                     'type': 'integer'},
                                           'size': {'title': 'Size',
                                                    'type': 'number'}},
                            'required': ['count'],
                            'title': 'FooBar',
                            'type': 'object'},
                 'Gender': {'description': 'An enumeration.',
                            'enum': ['male', 'female', 'other', 'not_given'],
                            'title': 'Gender',
                            'type': 'string'}},
 'properties': {'Gender': {'$ref': '#/definitions/Gender'},
                'foo_bar': {'$ref': '#/definitions/FooBar'},
                'snap': {'default': 42,
                         'description': 'this is the value of snap',
                         'exclusiveMaximum': 50,
                         'exclusiveMinimum': 30,
       

`schema()`返回字典，`schema_json()`返回json字符串：

In [29]:
print(MainModel.schema_json(indent=2))

{
  "title": "Main",
  "type": "object",
  "properties": {
    "foo_bar": {
      "$ref": "#/definitions/FooBar"
    },
    "Gender": {
      "$ref": "#/definitions/Gender"
    },
    "snap": {
      "title": "The Snap",
      "description": "this is the value of snap",
      "default": 42,
      "exclusiveMinimum": 30,
      "exclusiveMaximum": 50,
      "type": "integer"
    }
  },
  "required": [
    "foo_bar"
  ],
  "definitions": {
    "FooBar": {
      "title": "FooBar",
      "type": "object",
      "properties": {
        "count": {
          "title": "Count",
          "type": "integer"
        },
        "size": {
          "title": "Size",
          "type": "number"
        }
      },
      "required": [
        "count"
      ]
    },
    "Gender": {
      "title": "Gender",
      "description": "An enumeration.",
      "enum": [
        "male",
        "female",
        "other",
        "not_given"
      ],
      "type": "string"
    }
  }
}


有关schema还有很多内容，但这方面内容平时可能使用比较少，因此省略，留待补充。

## 导出模型

### model.dict(...)

模型转换成字典，接受以下参数：
- include：包含在返回的字典中的字段。参见《包含和排序的高级用法》。
- exclude：从返回的字典中排除的字段。参见《包含和排序的高级用法》。
- by_alias：字段别名是否应该在返回的字典中作为键，默认为 False。
- exclude_unset：创建模型时未显式设置的字段是否应从返回的字典中排除，默认为 False。在v1.0之前，exclude_unset被称skip_defaults；现在不赞成使用skip_defaults。
- exclude_defaults：是否应从返回的字典中排除等于其默认值的字段 (无论是否设置)，默认为 False。
- exclude_none：是否应从返回的字典中排除等于 None 的字段，默认为False。

这种方法嵌套的模型也会被转换成字典。

In [49]:
from pydantic import BaseModel


class BarModel(BaseModel):
    whatever: int


class FooBarModel(BaseModel):
    banana: float
    foo: str
    bar: BarModel


m = FooBarModel(banana=3.14, foo='hello', bar={'whatever': 123})

print(m.dict())
print(m.dict(include={'foo', 'bar'}))
print(m.dict(exclude={'foo', 'bar'}))

{'banana': 3.14, 'foo': 'hello', 'bar': {'whatever': 123}}
{'foo': 'hello', 'bar': {'whatever': 123}}
{'banana': 3.14}


### dict(model)和迭代

可以直接使用内置的`dict`转换模型，也可以直接对模型进行迭代，就像是迭代一个字典一样。但是此时嵌套的子模型不会转换成字典：

In [50]:
from pydantic import BaseModel


class BarModel(BaseModel):
    whatever: int


class FooBarModel(BaseModel):
    banana: float
    foo: str
    bar: BarModel


m = FooBarModel(banana=3.14, foo='hello', bar={'whatever': 123})

print(dict(m))
for name, value in m:
    print(f'{name}: {value}')

{'banana': 3.14, 'foo': 'hello', 'bar': BarModel(whatever=123)}
banana: 3.14
foo: hello
bar: whatever=123


### model.copy(...)

`copy`方法可以复制模型，接受以下参数：
- include：要包含在返回的字典中的字段。参见《包含和排序的高级用法》
- exclude：要从返回的字典中排序的字典。参见《包含和排序的高级用法》。
- update：创建复制的模型时要更改的值的字典。
- deep：是否对新模型进行深复制；默认为 False。

In [51]:
from pydantic import BaseModel


class BarModel(BaseModel):
    whatever: int


class FooBarModel(BaseModel):
    banana: float
    foo: str
    bar: BarModel


m = FooBarModel(banana=3.14, foo='hello', bar={'whatever': 123})

print(m.copy(include={'foo', 'bar'}))
print(m.copy(exclude={'foo', 'bar'}))
print(m.copy(update={'banana': 0}))
print(id(m.bar), id(m.copy().bar))  # 普通的copy，copy的是子模型的引用
print(id(m.bar), id(m.copy(deep=True).bar))  # 深度copy，创建了一个新的子模型

foo='hello' bar=BarModel(whatever=123)
banana=3.14
banana=0 foo='hello' bar=BarModel(whatever=123)
2009639472768 2009639472768
2009639472768 2009651214368


### model.json(...)

`.json()`方法会将模型序列化为JSON。通常，在内部，`.json()`依次调用`.dict()`并序列化其结果。因此它接收的参数和`.dict()`方法基本一致，增加的参数如下：
- `encoder`：传递给 json.dumps() 的 default 参数的自义编码器函数，默认为设计用于所有常见类型的自定义编码器。
- `**dumps_kwargs`：传递给 json.dumps() 的其他关键字参数。例如，indent。

#### json.encoders

序列化可以使用`json_encoders`配置属性在模型上定制，键应该是类型，值应该是序列化该类型的函数：

In [9]:
from datetime import datetime, timedelta
from pydantic import BaseModel
from pydantic.json import timedelta_isoformat


class WithCustomEncoders(BaseModel):
    dt: datetime
    diff: timedelta

    class Config:
        json_encoders = {
            datetime: lambda v: v.timestamp(),
            timedelta: timedelta_isoformat,
        }


m = WithCustomEncoders(dt=datetime(2032, 6, 1), diff=timedelta(hours=100))
print(m.json())

{"dt": 1969632000.0, "diff": "P4DT4H0M0.000000S"}


#### 自定义json反序列化

为了提高json解码编码的性能，我们有时候可以通过可以配置中的`json_load`和`json_dumps`属性来指定其它的json实现：

In [52]:
from datetime import datetime
import ujson
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None

    class Config:
        json_loads = ujson.loads

In [53]:
user = User.parse_raw('{"id": 123,"signup_ts":1234567890,"name":"John Doe"}')
print(user)

id=123 signup_ts=datetime.datetime(2009, 2, 13, 23, 31, 30, tzinfo=datetime.timezone.utc) name='John Doe'


但是ujson通常不能用于转储JSON，因为它不支持对像`datetime`这样的对象进行编码，也不接受`default`回退函数参数：

In [54]:
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None

    class Config:
        json_loads = ujson.loads
        json_dumps = ujson.dumps

In [58]:
user = User.parse_raw('{"id": 123,"signup_ts":1234567890,"name":"John Doe"}')
try:
    user.json()
except TypeError as e:
    print(e)

'default' is an invalid keyword argument for this function


此时可以不指定`json_dumps`，也可以使用orjson：

In [69]:
from datetime import datetime
import orjson
from pydantic import BaseModel


def orjson_dumps(v, *, default):
    # orjson.dumps returns bytes, to match standard json.dumps we need to decode
    return orjson.dumps(v, default=default).decode()


class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: datetime = None

    class Config:
        json_loads = orjson.loads
        json_dumps = orjson_dumps


user = User.parse_raw('{"id":123,"signup_ts":1234567890,"name":"John Doe"}')
print(user.json())  # 此时就不能接收比如indent等参数了

{"id":123,"signup_ts":"2009-02-13T23:31:30+00:00","name":"John Doe"}


### `include`和`exclude`

`include`和`exclude`可以指定包含或者排除的字段，但是有几点要注意。

In [71]:
from pydantic import BaseModel, SecretStr


class User(BaseModel):
    id: int
    username: str
    password: SecretStr


class Transaction(BaseModel):
    id: str
    user: User
    value: int


t = Transaction(
    id='1234567890',
    user=User(
        id=42,
        username='JohnDoe',
        password='hashedpassword'
    ),
    value=9876543210,
)

# 没有嵌套模型的情况下我们可以直接使用集合
print(t.dict(exclude={'user', 'value'}))

# 如果有嵌套的模型，测试使用字典，此时值是普通类型的字段，比如value，字典中的值可以设置为省略号
print(t.dict(exclude={'user': {'username', 'password'}, 'value': ...}))
print(t.dict(include={'id': ..., 'user': {'id'}}))

{'id': '1234567890'}
{'id': '1234567890', 'user': {'id': 42}}
{'id': '1234567890', 'user': {'id': 42}}


如果子模型或者字段的值是序列，我们想排除或者包含其中某部分值怎么办呢？可以像索引一样，直接使用数字来选取：

In [3]:
import datetime
from typing import List
from pprint import pprint

from pydantic import BaseModel, SecretStr


class Country(BaseModel):
    name: str
    phone_code: int


class Address(BaseModel):
    post_code: int
    country: Country


class CardDetails(BaseModel):
    number: SecretStr
    expires: datetime.date


class Hobby(BaseModel):
    name: str
    info: str


class User(BaseModel):
    first_name: str
    second_name: str
    address: Address
    card_details: CardDetails
    hobbies: List[Hobby]


user = User(
    first_name='John',
    second_name='Doe',
    address=Address(
        post_code=123456,
        country=Country(
            name='USA',
            phone_code=1
        )
    ),
    card_details=CardDetails(
        number=4212934504460000,
        expires=datetime.date(2020, 5, 1)
    ),
    hobbies=[
        Hobby(name='Programming', info='Writing code and stuff'),
        Hobby(name='Gaming', info='Hell Yeah!!!'),
    ],
)

In [7]:
include_keys = {
    'first_name': ...,
    'address': {'country': {'name'}},
    'hobbies': {0: ..., -1: {'name'}},
}

pprint(user.dict(include=include_keys))

{'address': {'country': {'name': 'USA'}},
 'first_name': 'John',
 'hobbies': [{'info': 'Writing code and stuff', 'name': 'Programming'},
             {'name': 'Gaming'}]}


如果序列里所有的项都要排除或者选取，可以使用`__all__`作为键：

In [9]:
# To exclude a field from all members of a nested list or tuple, use "__all__":
pprint(user.dict(exclude={'hobbies': {'__all__': {'info'}}}))

{'address': {'country': {'name': 'USA', 'phone_code': 1}, 'post_code': 123456},
 'card_details': {'expires': datetime.date(2020, 5, 1),
                  'number': SecretStr('**********')},
 'first_name': 'John',
 'hobbies': [{'name': 'Programming'}, {'name': 'Gaming'}],
 'second_name': 'Doe'}


## 代码生成

`datamodel-code-generator`是一个插件，它甚至能够根据已有的json和yaml等几乎任何数据源，自动生成pydantic模型。
- [官网地址](https://koxudaxi.github.io/datamodel-code-generator/)

使用很简单，运行`pip install datamodel-code-generator`安装以后，命令行运行：
```
datamodel-codegen --input api.yaml --output model.py
```
即可把文件生成相应的模型。